# Import the required package

In [1]:
import os
import sys
import pandas as pd
import py4cytoscape as p4c
%matplotlib inline

# Setup Cytoscape

In [2]:
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.8.2',
 'automationAPIVersion': '1.0.0',
 'py4cytoscapeVersion': '0.0.6'}

# Import Network and Data

In [3]:
ap_ms_url = 'https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/R/notebooks/AP-MS/ap-ms-demodata.csv'
ap_ms_df = pd.read_csv(ap_ms_url)
ap_ms_df.head()

,Bait,Prey,UniProt,HEKScore,JurkatScore,AP-MS Score
0,GAG,THRAP3,Q9Y2W1,0.807,0.319,0.563
1,GAG,SEPSECS,Q9HD40,0.814,0.200,0.507
2,GAG,IVNS1ABP,Q9Y6Y0,0.753,0.260,0.506
3,GAG,DDX49,Q9Y6V7,0.824,0.000,0.412
4,GAG,PRMT1,Q99873,0.758,0.037,0.397


In [4]:
edge_data = {'source':ap_ms_df["Bait"],
             'target':ap_ms_df["Prey"],
             'interaction': "AP-MS",
             'AP-MS Score':ap_ms_df["AP-MS Score"]
            }
edges = pd.DataFrame(data=edge_data, columns=['source', 'target', 'interaction','AP-MS Score'])
edges.head(10)

,source,target,interaction,AP-MS Score
0,GAG,THRAP3,AP-MS,0.563
1,GAG,SEPSECS,AP-MS,0.507
2,GAG,IVNS1ABP,AP-MS,0.506
3,GAG,DDX49,AP-MS,0.412
4,GAG,PRMT1,AP-MS,0.397
5,GAG,OLA1,AP-MS,0.389
6,GAG,EIF2AK2,AP-MS,0.383
7,GAG,RBM17,AP-MS,0.382
8,MA,EPRS,AP-MS,0.874
9,MA,OLA1,AP-MS,0.865


In [5]:
p4c.networks.create_network_from_data_frames(None, edges, title='apms network', collection = "apms collection")

Applying default style...
Applying preferred layout


{'networkSUID': 51}

# Loading Data

In [6]:
p4c.tables.load_table_data(ap_ms_df.iloc[:, 1:5], data_key_column="Prey")

'Success: Data loaded in defaultnode table'

# Augmenting Network with Existing Protein-protein Interaction Data

In [7]:
uniport_str = ','.join(ap_ms_df["UniProt"])

In [8]:
string_cmd_list = ['string protein query','query="',uniport_str,'"', 'species="Homo sapiens"', 'limit=0', 'cutoff=0.999']

In [9]:
string_cmd = " ".join(string_cmd_list)

In [10]:
p4c.commands.commands_run(string_cmd)

["Loaded network 'String Network' with 434 nodes and 302 edges"]

# Merge Networks

In [11]:
p4c.tools.merge_networks(["apms network,String Network"], operation='union', node_keys=["Uniprot,query term"], node_merge_map=[["name","display name","display name", "string"]])

3481

# Network Visualization

In [12]:
p4c.networks.set_current_network('union: apms network,String Network')
p4c.network_views.set_current_view()

{}

In [13]:
style_name = "AP-MS"
p4c.styles.create_visual_style(style_name)
p4c.style_dependencies.lock_node_dimensions(True)
p4c.style_defaults.set_node_size_default(50, style_name=style_name)
p4c.style_defaults.set_node_color_default("#CCCCFF", style_name=style_name)
p4c.style_mappings.set_node_label_mapping('display name', style_name=style_name)
p4c.styles.set_visual_style(style_name=style_name)

style_name not specified; updating "default" style.


{'message': 'Visual Style applied.'}

In [14]:
p4c.layouts.layout_network(layout_name='force-directed defaultSpringCoefficient=0.00001 defaultSpringLength=50 defaultNodeMass=4')

{}

# STRING Enrichment

In [15]:
p4c.commands.commands_run('string make string network="current"')

["Set network 'union: apms network,String Network as STRING network"]

In [16]:
string_cmd_list = ['string retrieve enrichment','allNetSpecies="Homo sapiens"', 'background="genome"','selectedNodesOnly="false"']
string_cmd = " ".join(string_cmd_list)
p4c.commands.commands_run(string_cmd)

['Enrichment results summary:',
 'ppiEnrichment=1.0E-16',
 'enrichmentNodes=434',
 'enrichmentExpectedEdges=25',
 'enrichmentEdges=302',
 'enrichmentClusteringCoeff=0.324709238534123',
 'enrichmentAvgDegree=1.39170506912442']

In [17]:
p4c.commands.commands_run('string filter enrichment categories="GO Process" removeOverlapping="true"')

['']

In [18]:
p4c.commands.commands_run('string show charts')

[]

# Visualizing Results - Jurkat Score

In [19]:
style_name = "AP-MS Jurkat Score"
p4c.styles.create_visual_style(style_name)
p4c.styles.set_visual_style(style_name=style_name)
p4c.style_defaults.set_node_color_default("#FFCC00", style_name=style_name)
p4c.style_mappings.set_node_label_mapping('display name', style_name=style_name)
p4c.style_mappings.set_edge_color_mapping("interaction", ["AP-MS"], ["#55AA55"], "d", style_name=style_name)
p4c.style_mappings.set_edge_line_width_mapping('AP-MS Score', table_column_values=[0, 1], widths=[1,5], mapping_type="c", style_name=style_name)

''

In [20]:
data_values = [0, 0.5, 1]
node_colors = ["#756BB1", "#BCBDDC", "#EFEDF5"]
p4c.style_mappings.set_node_color_mapping('JurkatScore', data_values, node_colors, style_name=style_name)

''

# Visualizing Results - Combined

In [21]:
p4c.styles.copy_visual_style("AP-MS Jurkat Score", "AP-MS CombinedScore")
p4c.styles.set_visual_style(style_name="AP-MS CombinedScore")

{'message': 'Visual Style applied.'}

In [22]:
all_nodes = p4c.networks.get_all_nodes()

In [23]:
combined_df = pd.DataFrame(all_nodes, columns=['name'])
combined_df["CombinedScore"] = 'piechart: showlabels=false range="0,1" arcstart=90 valuelist=".5,.5" colorlist="up:blue,zero:white,down:white;up:purple,zero:white,down:white" attributelist="HEKScore,JurkatScore"'

In [24]:
p4c.tables.load_table_data(combined_df, data_key_column="name", table_key_column='name',)

'Success: Data loaded in defaultnode table'

In [25]:
piechart_map = p4c.style_mappings.map_visual_property('node customgraphics 4','CombinedScore','p')
p4c.style_mappings.update_style_mapping('AP-MS CombinedScore', piechart_map)

''

# Saving, sharing and publishing

In [26]:
p4c.session.save_session('AP-MS_session')

{}

# Saving high resolution image files

In [27]:
p4c.network_views.export_image('AP-MS_image', type = 'PDF')

{'file': 'C:\\Users\\YihangXin\\CytoscapeConfiguration\\filetransfer\\default_sandbox\\AP-MS_image.pdf'}